ローソク足作成
- bybitから約定データを取得･保存
- 約定データからローソク足を作成

In [1]:
from datetime import datetime
from datetime import timedelta
import gzip
import os

from tqdm import tqdm
import pandas as pd
from urllib import request

## bybit api操作基礎
https://note.com/kanawoinvestment/n/ne287ff725fd9

## 約定データの作成

対象ペア    
'BTC/USD'    
'ETH/USD'    
'EOS/USD'    
'XRP/USD'    
'BTC/USDT'    
'BCH/USDT'    
'ETH/USDT'    
'LTC/USDT'    
'LINK/USDT'    
'XTZ/USDT'   

In [2]:
def create_date_list(start_date:str,
                     end_date:str) -> list:
    """
    文字列型の日付リストを作成
    """
    datetime_format = '%Y-%m-%d'
    start = datetime.strptime(start_date, datetime_format)
    end = datetime.strptime(end_date, datetime_format)
    period = (end - start).days
    return [datetime.strftime(start+timedelta(i),
                               datetime_format)
            for i in range(period+1)]

def fetch_executions(target_zipfile_url:str) -> pd.DataFrame:
    # URL先ファイルを取得
    request.urlretrieve(target_zipfile_url,'tmp_zip.csv.gz')
    with gzip.open('tmp_zip.csv.gz', 'rb') as f:
        df = pd.read_csv(f)
    return df

In [9]:
# 対象通貨ペア指定
SYMBOL = 'BTCUSD'
# bybitの約定CSVデータの保存先
BASE_URL = 'https://public.bybit.com/trading/'+SYMBOL+'/'
# 約定データの保存先
DATA_DIR = '../../data/crypto/bybit/executions/'+SYMBOL+'/'

# 回収対象期間
start_date = '2019-10-01'
end_date = '2021-03-13'

In [10]:
date_list = create_date_list(start_date,
                             end_date)

for date in tqdm(date_list):
    try:
        csvfile_name = SYMBOL+date+'.csv'
        zipfile_name = csvfile_name + '.gz'
        target_zipfile_url = BASE_URL + zipfile_name
        df_exec = fetch_executions(target_zipfile_url)
        df_exec.to_csv(DATA_DIR+csvfile_name)
    except:
        continue
        
os.remove('tmp_zip.csv.gz')

100%|██████████| 530/530 [41:59<00:00,  4.75s/it]


## ローソク足作成
参考

In [13]:
# 対象ペア
SYMBOL = 'ETHUSD'

start_date = '2019-10-01'
end_date   = '2021-03-13'

date_list = create_date_list(start_date,
                             end_date)

In [14]:
durations = [
    '1min',
    '5min',
    '15min',
    '30min',
    '1H',
    '4H',
    '1D',
    '1W',
    '1M',
]

# D: 毎日
# H: 毎時
# Tまたはmin: 毎分
# S: 毎秒
# Lまたはms: 毎ミリ秒
# Uまたはus: 毎マイクロ秒
# N: 毎ナノ秒
# 参考: https://note.nkmk.me/python-pandas-time-series-freq/

for duration in durations:
    ohlcv_list = []
    for date in tqdm(date_list):
        try:
            df_exec = pd.read_csv(
                '../../data/crypto/bybit/executions/'
                + SYMBOL + '/'
                + SYMBOL + date + '.csv')
            df_exec['exec_timestamp'] = pd.to_datetime(df_exec['timestamp'],
                                                       unit='s')
            df_exec = df_exec.set_index('exec_timestamp')[['side',
                                                           'size',
                                                           'price']]
            df_ohlcv = df_exec.resample(duration).agg({'price': 'ohlc',
                                                       'size' : 'sum'}).ffill()
            df_ohlcv.columns = ['open','high','low','close','volume']
            ohlcv_list.append(df_ohlcv)

        except:
            continue
            
    pd.concat(ohlcv_list).to_csv(
        '../../data/crypto/bybit/ohlcv/'
        + SYMBOL + '/'
        + SYMBOL + '_' + duration + '.csv')

100%|██████████| 530/530 [04:56<00:00,  1.79it/s]


## 売買別出来高を集計

In [ ]:
# TODO

## 板情報の集計

In [ ]:
# TODO

In [15]:
import ccxt

bybit = ccxt.bybit()

In [16]:
bybit.fetch_order_book('BTC/USD')

{'bids': [[54038.5, 3214770.0],
  [54038.0, 370150.0],
  [54037.5, 320148.0],
  [54037.0, 271983.0],
  [54036.5, 126541.0],
  [54036.0, 130021.0],
  [54035.5, 188.0],
  [54034.5, 634028.0],
  [54034.0, 129201.0],
  [54033.5, 20241.0],
  [54033.0, 17290.0],
  [54032.5, 97105.0],
  [54032.0, 86678.0],
  [54031.5, 48216.0],
  [54031.0, 44865.0],
  [54030.5, 29613.0],
  [54030.0, 17937.0],
  [54029.5, 34461.0],
  [54029.0, 74059.0],
  [54028.5, 203195.0],
  [54028.0, 154414.0],
  [54027.5, 219500.0],
  [54027.0, 170890.0],
  [54026.5, 10334.0],
  [54026.0, 132393.0]],
 'asks': [[54039.0, 161257.0],
  [54039.5, 915.0],
  [54040.0, 4984.0],
  [54040.5, 4361.0],
  [54041.0, 1831.0],
  [54041.5, 3.0],
  [54042.0, 1660.0],
  [54042.5, 10.0],
  [54043.5, 11.0],
  [54044.0, 5033.0],
  [54044.5, 9.0],
  [54045.5, 1831.0],
  [54046.0, 18672.0],
  [54047.0, 321.0],
  [54047.5, 1277.0],
  [54048.0, 2006.0],
  [54048.5, 75.0],
  [54049.5, 2005.0],
  [54050.0, 30025.0],
  [54050.5, 200.0],
  [54051.0, 